In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

## Key inputs

In [2]:
filename = ('SampleData.csv')
role_lookup = ('') 

In [3]:
data = pd.read_csv(filename)

# Basic cleanup

In [4]:
# fix columns data types in python
data['CaseId'] = data['CaseId'].astype('str')

# convert datetime column and get the dd-mm right
data['StartTimestamp'] = pd.to_datetime(data['StartTimestamp'], dayfirst=True)
data['EndTimestamp'] = pd.to_datetime(data['EndTimestamp'], dayfirst=True)

# Create a list of unique CaseIds
app_numbers = data['CaseId'].drop_duplicates().tolist()

## 1. Create a new raw for a starting activity
Not needed if the data don't have "Start Time Stamp"

In [5]:
# 1. Add an extra raw for each application as a start activity
for app in app_numbers:
    app_events = data[data['CaseId']==app]
    
    #Sort events based on StartTimestamp
    app_events = app_events.sort_values (by='StartTimestamp', ascending = True)
    
    # Duplicate first raw to dataset
    row_to_duplicate = app_events.iloc[0]
    data = data.append(row_to_duplicate, ignore_index=True)
    
    # Get and sort app events again
    app_events = data[data['CaseId']==app]
    
    #Sort events based on StartTimestamp
    app_events = app_events.sort_values (by='StartTimestamp', ascending = True)    
    
    # Replace values in the newly created row
    new_index = app_events.index[0]
    new_activity_name = 'start'
    new_role_name = 'start'
    new_end_timestamp = app_events ['StartTimestamp'].iloc[0]
    
    data.at[data.index[new_index],'ActivityName'] = new_activity_name
    data.at[data.index[new_index],'EndTimestamp'] = new_end_timestamp
    data.at[data.index[new_index],'Role'] = new_role_name

In [6]:
# Test previous code
data[data['CaseId']=='1'].sort_values(by='StartTimestamp', ascending = True)

,Location,Role,VendorID,InvoiceValue,Resource,StartTimestamp,EndTimestamp,CaseId,ActivityName
5,"San Francisco, USA",start,Vendor2,2499.1,Aiden,2022-07-21 09:43:01.043627,2022-07-21 09:43:01.043627,1,start
1005,"San Francisco, USA",Invoice Processor,Vendor2,2499.1,Aiden,2022-07-21 09:43:01.043627,2022-07-21 12:05:24.443627,1,Invoice Entry
6,"San Francisco, USA",Invoice Processor,Vendor2,2499.1,Aiden,2022-07-21 12:05:24.443627,2022-07-21 13:54:26.243627,1,Check Customer Payment
7,"San Francisco, USA",Invoice Processor,Vendor2,2499.1,Aiden,2022-07-21 13:54:26.243627,2022-07-21 15:44:38.843627,1,Credit Memo Entry
8,"San Francisco, USA",Invoice Processor,Vendor2,2499.1,Aiden,2022-07-21 15:44:38.843627,2022-07-21 17:51:44.243627,1,Refund Customer
9,"San Francisco, USA",Invoice Processor,Vendor2,2499.1,Aiden,2022-07-21 17:51:44.243627,2022-07-21 19:05:53.843627,1,Re-issuing the invoice


In [7]:
# Add "total step time (minutes)" calculated column
data ['TotalStepTime'] = data['EndTimestamp'] - data['StartTimestamp']

# Convert timedelta to minutes
data['TotalStepTime(minutes)'] = data ['TotalStepTime'].dt.total_seconds() / 60

data = data.drop('TotalStepTime', axis=1)

## 2. Create a lookup table of AHT estimates

these should be a way to simplify this or create the table in a separate spreadsheet

In [8]:
data['Role'].drop_duplicates()

0                 start
1     Invoice Processor
13     Invoice Approver
Name: Role, dtype: object

In [9]:
# Please input Average Handeling time in minutes (for every touch) - and the max processing time as multiple of the AHT

InvoiceProcessor_aht = 60
InvoiceApprover_aht = 30

max_multiply = 5

In [10]:
# Calculate the average TotalStepTime in dataset
InvoiceProcessor_atst = data[data['Role']=='Invoice Processor']['TotalStepTime(minutes)'].mean()
InvoiceApprover_atst = data[data['Role']=='Invoice Approver']['TotalStepTime(minutes)'].mean()

# Calculte the maximum processing time in minutes as a multiplyer of AHT
InvoiceProcessor_max = max_multiply * InvoiceProcessor_aht
InvoiceApprover_max = max_multiply * InvoiceApprover_aht

#Calculate te conversion rate and build a lookup table
lookup = [['Invoice Processor', InvoiceProcessor_atst/InvoiceProcessor_aht , InvoiceProcessor_max ],
         ['Invoice Approver', InvoiceApprover_atst/InvoiceApprover_aht , InvoiceApprover_max ]]

lookup = pd.DataFrame(lookup, columns=['Role' , 'ConversionConstant' , 'MaxValue'])

In [11]:
# Add lookup tablel to data
data = pd.merge(data, lookup, on='Role', how='left')

In [12]:
# Calculate the estimated processing time & round the outcome
data['EstimatedProcessingTime(minutes)'] = data['TotalStepTime(minutes)'] / data['ConversionConstant']
data['EstimatedProcessingTime(minutes)'] = data['EstimatedProcessingTime(minutes)'].round(2)

# Cap processing time at the threshould above
data['EstimatedProcessingTime(minutes)'] = data.apply(lambda row: min(row['MaxValue'], 
                                                                      row['EstimatedProcessingTime(minutes)']),axis=1)
# Drop extra columns
data = data.drop('ConversionConstant', axis=1)
data = data.drop('MaxValue', axis=1)

# Sort the dataset
data = data.sort_values(by='StartTimestamp', ascending=True)
data = data.sort_values(by='CaseId', ascending=True)

# Fill empty cells with 0
data['EstimatedProcessingTime(minutes)'] = data['EstimatedProcessingTime(minutes)'].fillna(0)

## 3. Create new Start Time Stamps

In [13]:
# Create a new process start timestamp considering the estimated processing times
data['ProcessingTimedelta'] = pd.to_timedelta(data['EstimatedProcessingTime(minutes)'], unit='m')
data['new_StartTimestamp'] = data['EndTimestamp'] - data['ProcessingTimedelta']

data=data.drop('ProcessingTimedelta', axis=1)

In [15]:
# Export output data
data.to_csv ('OutputDataset.csv', index=False)